In [3]:
import numpy as np

# **LAB 5:** Resolución de sistemas de ecuaciones lineales usando Gauss y Gauss-Jordan

El objetivo de esta práctica es programar los métodos de resolución directa de sistemas de ecuaciones lineales explicados en clase. 

Los programas deben recibir la matriz extendida del sistema $[A, b]$ (matriz de coeficientes y la última columna, la matriz de términos independientes).

**NOTA** Los métodos a implementar en las secciones 1 y 2 deben lanzar una Excepción si se recibe como entrada un sistema incompatible o compatible indeterminado.

Resuelve los siguientes sistemas:

1.  $\begin{cases}
    2x_1 + 4x_3 = 0\\
    6x_1 - 3x_2 + 7x_3 = 1\\
    -4x_1 + 6x_2 + 2x_3 = 3
    \end{cases}$

2. $\begin{cases}
    x_2 + x_3 = 1\\
    2x_1 + x_2 + 6x_3 - 3x_4 = 2\\
    -4x_1 -x_2 + 5x_3 + 8x_4 = 1\\
    -6x_1 -x_2 - 16x_3 + 13x_4 = 0
    \end{cases}$

3. $\begin{cases}
    6x_1 - 9x_2 + x_3 + 4x_5 = -5\\
    -2x_1 + 3x_2 - x_3 + 5x_4 + 9x_5 = -24\\
    x_2 - x_3 + 7x_4 + 2x_5 = -10\\
    5x_1 - 6x_2 + 8x_3 - x_4 = 5\\
    3x_1 + 7x_2 - 2x_3 + x_4 + 5x_5 = 2
    \end{cases}$

## 1. Método de Gauss

Implementa el método de Gauss para resolver sistemas de ecuaciones lineales explicado en teoría.

In [7]:
def gauss_method(M):
    # Comprobamos que el número de ecuaciones es igual al número de incógnitas
    if M.shape[0] != M.shape[1] - 1:
        raise ValueError("El sistema no tiene el mismo número de ecuaciones que de incógnitas")

    # Transformamos la matriz extendida en una matriz triangular superior
    for i in range(M.shape[0]):
        if M[i,i] == 0:
            # Si el elemento diagonal es cero, buscamos una fila donde el elemento no sea cero y la intercambiamos
            for j in range(i+1, M.shape[0]):
                if M[j,i] != 0:
                    M[[i,j],:] = M[[j,i],:]
                break
            else:
                # Si no encontramos una fila adecuada, el sistema es incompatible o compatible indeterminado
                raise ValueError("El sistema es incompatible o compatible indeterminado")

        # Dividimos la fila i por el elemento diagonal
        M[i,:] /= M[i,i]

        # Restamos la fila i a las filas siguientes para hacer cero los elementos debajo del diagonal
        for j in range(i+1, M.shape[0]):
            M[j,:] -= M[i,:] * M[j,i]   

    # Comprobamos que la matriz resultante es triangular superior
    if not np.allclose(np.triu(M[:,:-1]), M[:,:-1]):
        raise ValueError("El sistema es incompatible o compatible indeterminado")

    # Resolvemos el sistema por sustitución hacia atrás
    x = np.zeros(M.shape[0])
    for i in range(M.shape[0]-1, -1, -1):
        x[i] = M[i,-1] - np.dot(M[i,i+1:-1], x[i+1:])

    return x.tolist()

In [8]:
A = [[2, 0, 4,0], [6, -3, 7,1], [-4, 6, 2,3]]
x = gauss_method(A)
print(x)

[[ 5.62949953e+15]
 [ 4.69124961e+15]
 [-2.81474977e+15]]


In [10]:
A = [[0,1, 1,0,1], [2,1,6,-3,2], [-4,-1,5,8,1], [-6,-1,-16,13,0]]
x = gauss_method(A)
print(x)

[[1.6875]
 [0.875 ]
 [0.125 ]
 [1.    ]]


In [11]:
A = [[6,-9, 1,0,4,-5], [-2,3,-1,5,9,-24], [0,1,-1,7,2,-10], [5,-6,8,-1,0,5], [3,7,-2,1,5,2]]
x = gauss_method(A)
print(x)

[[ 2.]
 [ 1.]
 [ 0.]
 [-1.]
 [-2.]]


## 2. Método de Gauss-Jordan

Implementa el método de Gauss-Jordan para resolver sistemas de ecuaciones lineales.

In [6]:
def gauss_jordan_method(M):
    # Comprobamos que el número de ecuaciones es igual al número de incógnitas
    if M.shape[0] != M.shape[1] - 1:
        raise ValueError("El sistema no tiene el mismo número de ecuaciones que de incógnitas")

    # Transformamos la matriz aumentada en una forma reducida de fila escalonada
    for i in range(M.shape[0]):
        # Buscamos un pivote no nulo en la columna i
        pivot_row = i
        while pivot_row < M.shape[0] and M[pivot_row, i] == 0:
            pivot_row += 1

        if pivot_row == M.shape[0]:
            # Si no encontramos un pivote no nulo, el sistema es incompatible o compatible indeterminado
            raise ValueError("El sistema es incompatible o compatible indeterminado")

        if pivot_row != i:
            # Si el pivote no está en la fila i, intercambiamos las filas
            M[[i, pivot_row], :] = M[[pivot_row, i], :]

        # Dividimos la fila i por el pivote
        M[i, :] /= M[i, i]

        # Restamos la fila i a las filas diferentes de la i para hacer cero los elementos sobre y debajo del pivote
        for j in range(M.shape[0]):
            if j != i:
                M[j, :] -= M[j, i] * M[i, :]

    # Comprobamos que la matriz resultante es una forma reducida de fila escalonada
    if not np.allclose(np.diag(M[:, :-1]), np.ones(M.shape[0])):
        raise ValueError("El sistema es incompatible o compatible indeterminado")

    # Resolvemos el sistema por sustitución hacia atrás
    x = M[:, -1]

    return x.tolist()

In [7]:
A = [[2, 0, 4], [6, -3, 7], [-4, 6, 2]]
b = [0,1,3]
x = gauss_jordan_method(A, b)
print(x)

[-1.125899906842624e+16, -9382499223688534.0, 5629499534213120.0]


In [8]:
A = [[0,1, 1,0], [2,1,6,-3], [-4,-1,5,8], [-6,-1,-16,13]]
b = [1,2,1,0]
x = gauss_jordan_method(A, b)
print(x)

[1.6875000000000004, 0.875, 0.12499999999999997, 1.0000000000000002]


In [9]:
A = [[6,-9, 1,0,4], [-2,3,-1,5,9], [0,1,-1,7,2], [5,-6,8,-1,0], [3,7,-2,1,5]]
b = [-5,-24,-10,5,2]
x = gauss_jordan_method(A, b)
print(x)

[1.9999999999999998, 1.0, 1.1102230246251565e-16, -1.0, -2.0]


## 4. Cálculo de la inversa de una matriz

Programa una función que, dada una matriz $A$, devuelva su matriz inversa $A^{-1}$. Comprueba si funciona con la siguiente matriz:

$A = \begin{pmatrix}
    3 & -4 & 1\\
    2 & -4 & 1\\
    3 & -5 & 1
\end{pmatrix}$

In [10]:
def find_inverse(A):
    # Comprobamos que la matriz A es cuadrada
    if A.shape[0] != A.shape[1]:
        raise ValueError("La matriz debe ser cuadrada")

    # Creamos la matriz identidad del mismo tamaño que A
    I = np.identity(A.shape[0])

    # Concatenamos A e I para crear la matriz aumentada
    M = np.concatenate((A, I), axis=1)

    # Transformamos la matriz aumentada en una forma reducida de fila escalonada
    for i in range(M.shape[0]):
        # Buscamos un pivote no nulo en la columna i
        pivot_row = i
        while pivot_row < M.shape[0] and M[pivot_row, i] == 0:
            pivot_row += 1

        if pivot_row == M.shape[0]:
            # Si no encontramos un pivote no nulo, la matriz es singular y no tiene inversa
            raise ValueError("La matriz es singular y no tiene inversa")

        if pivot_row != i:
            # Si el pivote no está en la fila i, intercambiamos las filas
            M[[i, pivot_row], :] = M[[pivot_row, i], :]

        # Dividimos la fila i por el pivote
        M[i, :] /= M[i, i]

        # Restamos la fila i a las filas diferentes de la i para hacer cero los elementos sobre y debajo del pivote
        for j in range(M.shape[0]):
            if j != i:
                M[j, :] -= M[j, i] * M[i, :]

    # Extraemos la matriz inversa de la parte derecha de la matriz aumentada
    inv_A = M[:, A.shape[0]:]

    return inv_A

In [11]:
A = [[3,-4,1],[2,-4,1],[3,-5,1]]
inversa = find_inverse(A)
print(inversa)

[[0.9999999999999999, -0.9999999999999999, 0.0], [1.0, -2.775557561562892e-17, -1.0], [2.0000000000000004, 2.9999999999999996, -4.0]]
